Loading Libraries

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Embedding 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.preprocessing.text import one_hot 
from tensorflow.keras.layers import LSTM 
from tensorflow.keras.layers import Dense 
import nltk 
import re 
from nltk.corpus import stopwords 
### Dataset Preprocessing 
from nltk.stem.porter import PorterStemmer ##stemming purpose 



Loading dataset

In [2]:
df=pd.read_csv('/content/drive/MyDrive/NLP/Fake_News Classifier/LSTM_Stemm/train.csv')

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.shape

(20800, 5)

In [ ]:
df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [3]:
df=df.dropna()

In [ ]:
df.isna().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [ ]:
df.shape

(18285, 5)

In [ ]:
df['label'].value_counts()

0    10361
1     7924
Name: label, dtype: int64

Get independent Featyures

In [4]:

X=df.drop('label',axis=1)

Dependent Features

In [5]:
y=df['label']

In [6]:
tf.__version__

'2.8.2'

Vocabulary Size

In [7]:
voc_size=5000

In [8]:
message=X.copy()

In [ ]:
message.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [9]:
message.reset_index(inplace=True)

In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
ps = PorterStemmer() 
lm =nltk.WordNetLemmatizer
corpus=[] 
for i in range(0,len(message)): 
  review = re.sub('[^a-zA-Z]', ' ',message['title'][i]) 
  review = review.lower() 
  review = review.split() 
  review = [ps.stem(word) for word in review if not word in stopwords.words('english') ] 
  review = ' '.join(review) 
  corpus.append(review) 




In [12]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [13]:
one_hot_rep=[one_hot(word,voc_size) for word in corpus]
one_hot_rep[0]

[4598, 3896, 1807, 2577, 3762, 1186, 1582, 968, 2605, 4861]

Padding

In [14]:
sent_length=20 
embedded_docs=pad_sequences(one_hot_rep,padding='post',maxlen=sent_length) 
print(embedded_docs) 

[[4598 3896 1807 ...    0    0    0]
 [2711 3098   21 ...    0    0    0]
 [ 708 4020 2567 ...    0    0    0]
 ...
 [4904 4224  387 ...    0    0    0]
 [2229 2169 4302 ...    0    0    0]
 [2070 3864 1408 ...    0    0    0]]


In [15]:
embedded_docs[0]

array([4598, 3896, 1807, 2577, 3762, 1186, 1582,  968, 2605, 4861,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

Model Creation

In [17]:
embedding_vector_features=100 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 100)           500000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 580,501
Trainable params: 580,501
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
len(embedded_docs),y.shape

(18285, (18285,))

In [19]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [20]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

Model Traning

In [22]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 9s 9ms/step - loss: 0.3048 - accuracy: 0.8562 - val_loss: 0.2100 - val_accuracy: 0.9165
Epoch 2/10
192/192 [==============================] - 1s 7ms/step - loss: 0.1345 - accuracy: 0.9520 - val_loss: 0.2126 - val_accuracy: 0.9097
Epoch 3/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0935 - accuracy: 0.9676 - val_loss: 0.2345 - val_accuracy: 0.9155
Epoch 4/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0608 - accuracy: 0.9800 - val_loss: 0.2696 - val_accuracy: 0.9118
Epoch 5/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0439 - accuracy: 0.9873 - val_loss: 0.3470 - val_accuracy: 0.9082
Epoch 6/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0364 - accuracy: 0.9889 - val_loss: 0.3277 - val_accuracy: 0.9080
Epoch 7/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0290 - accuracy: 0.9916 - val_loss: 0.3717 - val_accuracy: 0.9047
Epoch 

Adding DropOuts

In [23]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

Performance Matrix and Accuracy

In [24]:
y_pred=model.predict(X_test)

In [34]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [35]:
from sklearn.metrics import confusion_matrix

In [36]:
confusion_matrix(y_test,y_pred)

array([[3419,    0],
       [2616,    0]])

In [37]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5665285832642917

In [38]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.57      1.00      0.72      3419
           1       0.00      0.00      0.00      2616

    accuracy                           0.57      6035
   macro avg       0.28      0.50      0.36      6035
weighted avg       0.32      0.57      0.41      6035



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
